In [1]:
library(ggplot2)
library(reshape)
library(RColorBrewer)
library(stringr)
library(dplyr)
library(reshape2)
library(grid)


Attaching package: ‘dplyr’


The following object is masked from ‘package:reshape’:

    rename


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following objects are masked from ‘package:reshape’:

    colsplit, melt, recast




In [11]:
library(doParallel)

registerDoParallel(cores=24)

In [ ]:
input_name <- paste0('01_935clock')
output_name <- paste0('predict_01_935clock')


file <- paste0('/disk0/sm/methyl/clock/candyCLOCK/DNAmAge_predict_', input_name, '.txt')
age_table <- read.table(file, sep='\t', header=TRUE)

sample <- read.csv('/disk0/sm/methyl/clock/data/sample_sheet.csv', header=TRUE, skip=6) %>% select(Sample_Name, Age, Gender) %>% rename(gender = Gender, age = Age)


algorithm <- c('Horvath', 'Hannum', 'Levine', 'skinHorvath', 'BLUP', 'EN', 'SM')
go <- age_table

all_diff <- cal_diff(go, algorithm, sample)


p <- make_plot(all_diff, output_name, c('id', 'age', 'gender'))
png(filename=paste0('/disk0/sm/methyl/clock/candyCLOCK/age_', output_name, '.png'),width=1200,height=600,unit="px",bg="transparent")
  grid.arrange(p, ncol=1, nrow=1)
dev.off()

mean(df_diff$SM)
sd(df_diff$SM)
mean(df_diff$SM^2)


In [2]:
age_final <- read.table("/disk0/sm/methyl/2024/03_pollution/total/data/age_algorithm.txt", sep="\t", fill = TRUE, header=TRUE)

# pheno <- read.csv('/disk0/sm/methyl/TEST/1031/test/test4_table.txt', sep='\t', header=T)
# target <- pheno %>% filter(Phenotype == 'Normal')
# age_final <- age_final %>% filter(id %in% target$ID)

m_age_final = melt(age_final)
head(m_age_final, 2)
m_age_final <- m_age_final %>% filter(variable != 'TL')
# summary(age_final)

Using id as id variables



,id,variable,value
,<chr>,<fct>,<dbl>
1,NIH23J4322396_11,Horvath,-3.485883
2,NIH23J4322396_1,Horvath,1.996862


In [6]:
colr <- c('#FB8072', '#FDB462', '#FFED6F', '#B3DE69', '#80B1D3', '#BC80BD', '#f363db', 'grey', 'black', 'white', 'grey30') # 

p <- ggplot(m_age_final, aes(x=variable, y=value, fill=variable)) + 
  # coord_cartesian(ylim = c(-20, 20)) +
  geom_hline(yintercept=0, color='red', linetype = 2) +
  geom_boxplot(outlier.shape = NA) + #
  labs(
        x = 'age predict method', 
        y = 'age difference') + 
  # scale_fill_manual(values=colr) +
  theme_bw() +
  theme(
    axis.text.x = element_text(size=18),
    axis.text.y = element_text(size=15),
    # plot.title = element_text(hjust = 0.5,size=10,face='bold'),
    axis.title.x = element_text(size=18),
    axis.title.y = element_text(size=18),
    axis.ticks.x = element_blank(), 
    legend.text = element_blank(), 
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70"),
    legend.title = element_blank(),
    legend.position = 'none',
  ) + 
  scale_fill_manual(values=colr) 
  # geom_jitter(shape=1, position=position_jitter(0.3), size=0.1) +
  #  geom_boxplot(width=0.1, outlier.shape = NA) 
  


  ggsave(p, file='/disk0/sm/methyl/clock/mydata/age_final.png', width=14000, height=9000, units='px', dpi=1000)
# dev.off()

In [125]:
a <- mata %>% filter(Phenotype == 'Normal')
summary(a$value)
b <- mata %>% filter(Phenotype == 'Patient')
summary(b$value)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-27.951  -2.783   1.121   1.046   4.551  16.874 

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
-15.54945  -3.28556  -0.04515   0.03119   3.34115  18.28779 

In [129]:
head(data)

,id,Horvath,Hannum,Levine,skinHorvath,BLUP,EN,PedBE,Wu,SM,Age,Gender,Phenotype
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
1,GSM1051525,-2.1708449,1.182595,-12.7130063,-7.5836311,-0.1323889,-3.249737,-54.32369,-55.97178,-2.2504515,67,f,Patient
2,GSM1051526,-1.4132539,4.515034,-3.9152476,0.8254934,-0.2908770,-1.798865,-37.97506,-39.94542,-0.7783173,49,f,Patient
3,GSM1051527,-0.7624145,6.243567,4.9978522,0.2355747,-0.2526004,1.576862,-40.38539,-42.43802,3.5379394,53,f,Patient
4,GSM1051528,0.2646223,5.365866,-3.5479203,0.4003199,-0.1873811,1.908896,-49.61215,-51.84854,4.5495976,62,f,Patient
5,GSM1051529,3.1417353,9.880847,-0.4593848,3.0470374,-0.5126756,3.218575,-23.83183,-24.53246,5.2332862,33,f,Patient
6,GSM1051530,-6.9891158,3.135387,-5.5477690,3.2865390,-0.3618541,1.365452,-44.68562,-48.80091,3.1132100,59,f,Patient


In [138]:
data <- merge(age_final, pheno, by.x='id', by.y='ID', all=T)
# head(data)
# data$group <- 'under40'
# data[data$Age >= 40, 'group'] = 'over40'
# mata <- melt(data, id.vars=c('id', 'group', 'Age'), measure.vars=c('SM'))
mata <- melt(data, id.vars=c('id', 'Gender'), measure.vars=c('Horvath'))
mata$Gender <- factor(mata$Gender, levels=c('f', 'm'), labels=c('Female', 'male'))
# mata$group <- factor(mata$group, levels=c('under40', 'over40'))
head(mata, 2)


,id,Gender,variable,value
,<chr>,<fct>,<fct>,<dbl>
1,GSM1051525,Female,Horvath,-2.170845
2,GSM1051526,Female,Horvath,-1.413254


In [140]:
colr <- c('#FB8072', '#80B1D3') # FFED6F FFED6 80B1D3

p <- ggplot(mata, aes(x=Gender, y=value, fill=Gender)) + 
  coord_cartesian(ylim = c(-50, 50)) +
  geom_hline(yintercept=0, color='red', linetype = 2) +
  geom_boxplot(outlier.shape = NA) + #
  labs(
        x = 'age predict method', 
        y = 'age difference') + 
  scale_fill_manual(values=colr) +
  theme_bw() +
  theme(
    axis.text.x = element_text(size=18),
    axis.text.y = element_text(size=15),
    # plot.title = element_text(hjust = 0.5,size=10,face='bold'),
    axis.title.x = element_text(size=18),
    axis.title.y = element_text(size=18),
    axis.ticks.x = element_blank(), 
    legend.text = element_blank(), 
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey70"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_line(linetype = "dotted", colour = "grey70"),
    legend.title = element_blank(),
    legend.position = 'none',
  ) 
 
  # geom_jitter(shape=1, position=position_jitter(0.3), size=0.1) +
  #  geom_boxplot(width=0.1, outlier.shape = NA) 
  


  ggsave(p, file='/disk0/sm/methyl/TEST/1031/result/try4_812/test4/age_final_T.png', width=9000, height=9000, units='px', dpi=1000)
# dev.off()

In [7]:
age <- read.table("/disk0/sm/methyl/TEST/1125_clock/grimAGE/age_table.txt", sep="\t", fill = TRUE, header=TRUE)

In [27]:
names(age)[11:19]

[1] "Horvath"     "Hannum"      "Levine"      "skinHorvath" "PedBE"      
[6] "Wu"          "BLUP"        "EN"          "SM"

In [28]:
algorithm <- names(age)[11:19]
for (i in algorithm){
  tmp <- age[c('Age', i)]
  names(tmp) <- c('Age', 'predict')
  print(paste(i, " : ", cor(tmp$Age, tmp$predict)))
}

[1] "Horvath  :  0.917667196041716"
[1] "Hannum  :  0.921301560984754"
[1] "Levine  :  0.899890638543212"
[1] "skinHorvath  :  0.956654109501993"
[1] "PedBE  :  0.844936781222854"
[1] "Wu  :  0.416248079106826"
[1] "BLUP  :  0.972033939095549"
[1] "EN  :  0.968346941894724"
[1] "SM  :  0.973836283090403"


In [29]:
algorithm <- names(age)[11:19]
for (i in algorithm){
  tmp <- age[c('Age', i)]
  names(tmp) <- c('Age', 'predict')
  grob3 <- grobTree(textGrob(paste("Coefficient : ", cor(tmp$Age, tmp$predict)), x = 0.03, y = 0.97, hjust = 0, gp = gpar(col = "#ff4848", fontsize = 17, fontface = "bold")))
  p <- ggplot(data=tmp, aes(x=Age, y=predict)) +
  coord_cartesian(xlim = c(0, 100), ylim = c(0, 100))  +
  scale_x_continuous(breaks = seq(0, 100, 10)) +
  scale_y_continuous(breaks = seq(0, 100, 10)) +
  geom_point(size=0.5) +
    labs(title = i, 
          x = 'actual age', 
          y = 'predicted age') + 
    theme_bw() +
    theme(
    #   axis.text.x = element_text(seq(0, 2198), angle=30, hjust=1),
      axis.text.x = element_text(size=15),
      axis.text.y = element_text(size=15),
      plot.title = element_text(hjust = 0.5,size=20,face='bold'),
      axis.title.x = element_text(size=18),
      axis.title.y = element_text(size=18),
      axis.ticks.x = element_blank(), 
      legend.text = element_text(size=10),
      
      panel.grid.major.x = element_line(linetype = "dotted", colour = "grey80"),
      panel.grid.major.y = element_line(linetype = "dotted", colour = "gray70"), 
      panel.grid.minor.x = element_blank(), 
      panel.grid.minor.y = element_blank(), 
      legend.title = element_blank()
      ) +
      geom_smooth(method=lm, se=F, color='#ff0055') +
      annotation_custom(grob3)
  #   scale_colour_manual(values=c('#ff4040')) 
  # ggsave(p, file='/disk0/sm/chip/plot/final/total.png', width=2833, height=1890, units='px')
  ggsave(p, file=paste0('/disk0/sm/methyl/TEST/1023/age_', i, '.png'), width=8000, height=8000, units='px', dpi=1000)
}

`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'
`geom_smooth()` using formula = 'y ~ x'


In [ ]:
####################

In [ ]:
library(minfi)
library(dplyr)
library(RColorBrewer)
library(ggplot2)
library(viridis)
library(reshape2)
library(data.table)

In [ ]:
!Sample_characteristics_ch1     "age: 34" 
!Sample_geo_accession   "GSM100

In [ ]:
raw_col <- '#00224D'
data_col <- '#E74646'
old_col <- '#66C2A5'

In [ ]:
# data <- read.table("/home/data/two-way_normalized_beta_value_all_CpGs.txt", sep="\t", fill=TRUE, header=TRUE, row.names=1)
data <- fread("/home/data/test/test_beta.txt", sep="\t", fill=TRUE, header=TRUE)[, 2:721]
head(data)

png(filename="/home/data/test/two-way_densityplot.png", width=800, height=500, unit="px")
densityPlot(as.matrix(data), sampGroups = NULL, main = "", xlab = "Beta", pal = data_col, add = TRUE, legend = TRUE)
dev.off()

In [ ]:
##########################

In [22]:
table <- read.table('/disk0/sm/methyl/clock/mydata/S678_annotation.txt', sep='\t', header=TRUE, fill=TRUE)

In [41]:
a <- data.frame(table(table$Relation_to_Island)) %>% filter(Var1 != '')
a$Var1 <- factor(a$Var1 , level=c('Island', 'Shore', 'Shelf', 'OpenSea'))
head(a)

,Var1,Freq
,<fct>,<int>
1,Island,204
2,OpenSea,225
3,Shelf,49
4,Shore,199


In [45]:
p <- ggplot(a, aes(x=Var1, y=Freq, fill=Var1))+
  geom_bar(stat = "identity", size=.2, width=0.6) +
#   coord_cartesian(ylim = c(0, 450000)) +
  # scale_y_continuous(breaks = seq(0, 1, 0.1), labels = paste0(seq(0, 100, 10), '%')) +
  labs(title = 'Annotation relative to CpG Islands') +
  theme_bw() +
  theme(
    axis.text.x = element_text(size=17),
    axis.text.y = element_text(size=15),
    plot.title = element_text(hjust = 0.5,size=20,face='bold'),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(), 
    legend.text = element_text(size=15),
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey90"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey80"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_blank(),
    legend.title = element_blank(),
    legend.position = 'none'
        ) +
    scale_fill_manual(values=c('#3754f5', '#79b3ff', '#fbb472', '#fbe272')) # '#4723e6', 
  
ggsave(p, file='/disk0/sm/methyl/clock/mydata/island.png', width=8000, height=6000, units='px', dpi=1000)

In [43]:
b <- data.frame(table(table$UCSC)) %>% filter(Var1 != '')
b$Var1 <- factor(b$Var1 , level=c('TSS1500', 'TSS200', '5UTR', 'exon', '3UTR'))
head(b)

,Var1,Freq
,<fct>,<int>
1,3UTR,12
2,5UTR,34
3,exon,94
4,TSS1500,134
5,TSS200,55


In [46]:
p <- ggplot(b, aes(x=Var1, y=Freq, fill=Var1))+
  geom_bar(stat = "identity", size=.2, width=0.6) +
#   coord_cartesian(ylim = c(0, 450000)) +
  # scale_y_continuous(breaks = seq(0, 1, 0.1), labels = paste0(seq(0, 100, 10), '%')) +
  labs(title = 'Functional annotation') +
  theme_bw() +
  theme(
    axis.text.x = element_text(size=17),
    axis.text.y = element_text(size=15),
    plot.title = element_text(hjust = 0.5,size=20,face='bold'),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(), 
    legend.text = element_text(size=15),
    
    panel.grid.major.x = element_line(linetype = "dotted", colour = "grey90"),
    panel.grid.major.y = element_line(linetype = "dotted", colour = "grey80"),
    panel.grid.minor.x = element_blank(), 
    panel.grid.minor.y = element_blank(),
    legend.title = element_blank(),
    legend.position = 'none'
        ) +
    scale_fill_manual(values=c('#19c736', '#72db48', '#f2ff79', '#ffd857', '#fbc072'))
  
ggsave(p, file='/disk0/sm/methyl/clock/mydata/region.png', width=8000, height=6000, units='px', dpi=1000)